In [1]:
!nvidia-smi

Wed Oct  6 13:37:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000001:00:00.0 Off |                    0 |
| N/A   28C    P0    34W / 250W |      0MiB / 16160MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [3]:
from train import *

In [4]:
args = get_args()

In [5]:
print(args)

Namespace(batch_size=16, checkpoint_dir='./checkpoints/', dataset_dir='./datasets/', dataset_mode='train', distributed=False, epochs=100, ff='/home/azureuser/.local/share/jupyter/runtime/kernel-d10af28b-d9ac-4701-8879-0de392df52eb.json', grid_size=5, init_type='xavier', init_variance=0.02, load_height=1024, load_width=768, local_rank=0, log_interval=100, memory_format='channels_last', ngf=64, no_lsgan=False, norm_G='spectralaliasinstance', num_upsampling_layers='most', project='VITON-HD', seed=3407, semantic_nc=13, shuffle=False, sync_bn=False, use_amp=False, use_wandb=False, workers=4)


In [6]:
args.batch_size = 8
args.use_amp = True
args.workers = 1

In [7]:
seed_everything(args.seed)

In [8]:
train_dataset = VITONDataset(args)
train_loader = VITONDataLoader(args, train_dataset)

In [14]:
%%time
# %lprun -f train_dataset.__getitem__ \
s = next(iter(train_loader.data_loader))
s = train_loader.device_augment(s, 'cuda', torch.channels_last)

CPU times: user 192 ms, sys: 140 ms, total: 332 ms
Wall time: 2 s


In [10]:
for k,v in s.items():
    print(k, v.shape)

parse_target_down torch.Size([8, 13, 256, 192])
parse_agnostic torch.Size([8, 13, 1024, 768])
pose torch.Size([8, 3, 1024, 768])
cloth torch.Size([8, 3, 1024, 768])
cloth_mask torch.Size([8, 1, 1024, 768])


In [11]:
# %%time
# %lprun -f train_dataset.__getitem__ \
# s = train_dataset[0]